<img src='logo.png' style='display: block;height: 61px;float: left;padding: .75rem 1.25rem;'>

# Desafío - Inferencia de tópicos con EM
* Para realizar este desafío debes haber revisado la lectura y videos correspondiente a la unidad.
* Crea una carpeta de trabajo y guarda todos los archivos correspondientes (notebook y csv).
* Una vez terminado el desafío, comprime la carpeta y sube el `.zip` a la seccióncorrespondiente.

## Descripción

* En esta sesión trabajaremos con una serie de base de datos sobre letras musicales de distintos artistas. Cada uno de los `csv` se encuentra en la carpeta `dump`.
* Cada `csv` tiene el nombre del artista a analizar. Los archivos contienen el nombre del artista, elgénero musical del artista, el nombre de la canción y las letras.
* En base a esta información, el objetivo del ejercicio es generar un modelo probabilístico quepueda identificar el género musical más probable dado la letra de una canción.
* Para ello implementaremos un modelo conocido como Latent Dirichlet Allocation que hace uso de una variante del algoritmo EM para inferir clases latentes a partir de una matriz dedocumentos.

## Ejercicio 1: Preparar el ambiente de trabajo

* Importe los módulos `numpy`, `pandas`, `matplotlib`, `seaborn`, `glob` y `os` siguiendo las buenas prácticas. Los últimos dos módulos permitirán realizar la importación de múltiplesarchivos dentro de la carpeta dump.
* Para ello genere un objeto que guarde en una lista todos los archivos alojados en dumputilizando `glob.glob` y `os.getcwd()` para extraer las rutas absolutas. Posteriormente genere un objeto `pd.DataFrame` que contenga todos los csv.
* Asegúrese de eliminar la columna `Unnamed: 0` que se genera por defecto.

> Se utilizan librerías bases para el tratamiento de datos y algunos aspectos básicos de cálculo y gráficos.

In [1]:
# Se importa libreria para el manejo de bases de datos
import pandas as pd
# Se importa libreria para el manejo de operaciones de cálculo
import numpy as np
# Se importa libreria para el manejo de gráficos
import matplotlib.pyplot as plt
# Se importa libreria para manejo de funciones estadisticas y probabilidades
import seaborn as sns
# Librería para visualizar patrones de datos perdidos
import missingno as msngo
# Se importa libreria para el menejo de warning
import warnings
# Se importa libreria para el menejo de expresiones regulares
import glob, os

> Definimos algunos aspectos de ambiente y valores por defecto.

In [3]:
# Por defecto, matplotlib crea una figura en una ventana separada.
# podemos hacer que las figuras aparezcan en línea dentro del notebook; lo hacemos ejecutando:
%matplotlib inline
# Se ignoran los warning para evitar suciedad en la ejecución
warnings.filterwarnings(action='ignore')
# Se define el estilo de gráficos a usar
plt.style.use('seaborn-pastel')
# Se define el tamaño de los paños de los gráficos por defecto
plt.rcParams['figure.figsize'] = (10, 6)
# Dado que vamos a supervisar datos no limitaremos la cantidad de columnas a mostrar en el despliegue del dataframe
pd.options.display.max_columns = None

> Se utiliza <strong>librería propia</strong> que continen funciones auxiliares, necesarias para el desarrollo de desafíos.

In [4]:
# Se importa libreria con funciones auxiliares
import ancilliary_funcs as afx

> Se utilizan librerías <strong>sklearn</strong> para el tratamiento de escala de valores de los atributos y seleccion de set de datos:

In [5]:
# Método para contar palabras y vectorizar
from sklearn.feature_extraction.text import CountVectorizer

> Carga de nuestra base de datos

In [6]:
# Generamos el listado de archivos con su path con la librería glob y lo
# almacenamos en la lista file_list
file_list = glob.glob(os.getcwd() + '/dump/*.csv')

In [7]:
# Iniciamos una lista vacia para cargar el contenido de cada archivo csv
append_csv = []
# Recorremos el listado de archivos, lo cargamos en memoria con read_csv y
# lo agregamos a la lista appappend_csv
for filename in file_list:
    append_csv.append(pd.read_csv(filename, index_col=None, header=0)
                      .drop(columns = 'Unnamed: 0')
                     )

In [8]:
# Se concatenan todos los objetos DataFrames almacenados en la lista append_csv en 1 solo DataFrame
df = pd.concat(append_csv)
# Se asigna un nombre a cada columna del DataFrame
df.columns = ['Artis', 'Genre', 'Song', 'Lyrics']

In [10]:
# Inspeccionamos los datos
df.sample(5)

,Artis,Genre,Song,Lyrics
204,Kiss,rock,When Lightning Strikes,"Alright \n It's my move, the ground's shakin' ..."
417,Eminem,hiphop,Rap Name - (Obie Trice),The rap game hip hop 101 the hardest 9 to 5 yo...
39,The Doors,rock,Celebration Of The Lizard (An Experiment/Work ...,Lions in the street and roaming \n Dogs in hea...
70,Iron Maiden,metal,The Prophecy,Now that I know that the right time has come \...
97,The Beatles,rock,Sgt. Pepper's Lonely Hearts Club Band,It was twenty years ago today \n That Sgt. Pep...


In [9]:
# Revisamos las dimensiones de la base de datos
df.shape

(9489, 4)

> La base de datos se compone de 9489 observaciones con 4 atributos. 

## Ejercicio 2: Matriz de ocurrencias

* Importe la clase `CountVectorizer` dentro de los módulos `feature_extraction.text` de lalibrería `sklearn`.
* Aplique la clase para extraer las 5000 palabras más repetidas en toda la base de datos.
* Con la clase inicializada, incorpore las letras con el método `fit_transform` y guarde los resultados en un nuevo objeto

> Convertimos los textos en vectores de recuento de palabras.

In [25]:
count_vectorizer = CountVectorizer(stop_words='english', max_features=5000)
count_vectorizer_fit = count_vectorizer.fit_transform(df['Lyrics'])

In [28]:
count_vectorizer_fit
words = count_vectorizer.get_feature_names()
len(words)

5000

In [35]:
type(count_vectorizer_fit)

scipy.sparse.csr.csr_matrix

In [21]:
words = count_vectorizer.get_feature_names()
words_count = count_vectorizer_fit.toarray().sum(axis=0)
df_words = pd.DataFrame({'word': words, 'count': words_count})
df_words_5000 = df_words.sort_values(by='count', ascending=False).head(5000)

In [22]:
df_words_5000.shape

(5000, 2)

In [24]:
df_words_5000.transpose()

24950  12859  23856  18439  23287  25479  48591  25165 29851  7009   \
word    like    don   know    got   just   love   yeah     ll    oh  cause   
count  19629  17398  14962  14171  13978  11268  11071  10028  9879   8356   

      43795 24735 8724  26067 1265  25969 3022  47086 37216  36025 38497  \
word   time   let  come   man   ain  make  baby  want   say  right  shit   
count  8037  8009  7866  7467  7323  6821  6735  6698  6684   6207  6128   

      46260 47265 24875  47080  18334 48695 15760 28852 17232 43070  29103  \
word     ve   way  life  wanna  gonna    yo  feel  need  fuck  tell  nigga   
count  5924  5713  5710   5356   5121  4859  4704  4579  4537  4533   4334   

      14110 48490 10951 2927   48229 18344  43452  29125 25341   25133 36900  \
word     em    ya   day  away  world  good  think  night  look  little  said   
count  4292  4222  4217  4126   4106  3910   3902   3835  3805    3708  3692   

      18004 20080  18456 48113   29108 18243 27329  19754 15299 34652  4460   \
word   girl   hey  gotta   won  niggas   god  mind  heart  eyes  real  black   
count  3618  3341   3272  3161    3057  3011  3000   2861  2857  2846   2832   

      41368 29000   4170  25313  4392    31547 30005   34677 19677  27807  \
word   stop   new  better  long  bitch  people   ooh  really  head  money   
count  2811  2803    2801  2781   2756    2739  2719    2681  2669   2551   

      25138 19744 20411 1202  15340 2519  4251  12035 20353 12054 3128  24080  \
word   live  hear  home    ah  face   ass   big   did  hold   die   bad    la   
count  2544  2529  2503  2476  2437  2421  2363  2359  2326  2303  2217  2202   

        43449 36670 44828 36254 45178  24911 44953 10993  24571 20258 19457  \
word   things   run   try  rock    uh  light  turn  dead  leave   hit  hard   
count    2186  2176  2165  2165  2146   2101  2078  2076   2056  2048  2037   

      18326    44032 43768 20131 5354  40164   22016      14892  11041 24615  \
word   gone  tonight  till  high   boy  soul  inside  everybody  death  left   
count  2016     2010  2003  1989  1971  1921    1915       1903   1897  1895   

       43442 14305 41068 32316  4727  48655    3948    29111 29899 19906  \
word   thing   end  stay  play  blood   yes  believe  niggaz   old  hell   
count   1882  1829  1828  1822   1792  1784     1768    1757  1727  1714   

      19337  18288  32235 17514 23640 42722  5646  4927  46993 6473   40919  \
word   hand  going  place  game  kill  talk  bring  body  walk  came  stand   
count  1700   1675   1632  1623  1619  1581   1577  1574  1557  1549   1524   

       47196  19377 43974 15455 30758 25725 17002 3735     43516   8751   \
word   watch  hands  told  fall  pain    ma  free  beat  thought  coming   
count   1496   1490  1483  1478  1475  1466  1466  1464     1426    1420   

       48387 25424 48201  5502  40077 4136    17244 42022 19135  10033 40079  \
word   wrong  lost  word  break   son  best  fuckin   sun    ha  crazy  song   
count   1411  1410  1401   1401  1394  1390    1389  1378  1375   1371  1369   

      16566 8572  34434 36324 48212 12051 20688 44786 19583  10768  48756  \
word    fly  cold   rap  roll  work  didn   hot  true  hate  dance  young   
count  1350  1344  1318  1316  1311  1309  1295  1286  1284   1284   1280   

      37814 35984  8734  18322 10751  41001 26675 5305     16753 6735   13176  \
word   seen  ride  comes   gon  damn  start  mean  bout  forever  care  dream   
count  1276  1275   1268  1260  1248   1247  1240  1232     1208  1200   1199   

        41523 47095  16001 15540  47647 10965 39267  36499 25021    4394   \
word   street   war  fight   far  white  days   sky  round  line  bitches   
count    1190  1187   1177  1176   1175  1170  1169   1165  1165     1159   

      46960 48004  19746    17868    17096  34644 20812  18010 44055 4790   \
word   wait   wit  heard  getting  friends  ready   huh  girls  took  blow   
count  1158  1151   1148     1138     1133   1130  1129   1119  1118  

## Ejercicio 3: Entrenamiento del Modelo

* Importe `sklearn.decomposition.LatentDirichletAllocatin` y `sklearn.model_selection.GridSearchCV`.
* Genere una búsqueda de grilla con los siguientes hiperparámetros:
  - n_components: [5, 10, 15].
  - learning_decay: [0.7, 0.5].
* Entrene la búsqueda de grilla con las letras en un formato vectorizado con CountVectorizer.
* Reporte brevemente cuál es la mejor combinación de hiperparámetros.

> Digresión: Latent Dirichlet Allocation

In [30]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV

In [41]:
searc_params = {
    'n_components': [5, 10, 15],
    'learning_decay': [0.5, 0.7]
}

In [42]:
cv_lda_model = GridSearchCV(
    LatentDirichletAllocation(learning_method = 'online'), 
    param_grid = searc_params
).fit(count_vectorizer_fit)

In [43]:
cv_lda_model.best_params_

{'learning_decay': 0.7, 'n_components': 5}

In [44]:
best_lda = cv_lda_model.best_estimator_

In [49]:
for topic_id, topic_name in enumerate(best_lda.components_):
    print(f"Tópico: {topic_id + 1}")
    concat_words_in_topics = [count_vectorizer.get_feature_names()[i] for i in topic_name.argsort()[:-15 -1: -1]]
    print(" ".join(concat_words_in_topics))

Tópico: 1
like fuck ain got shit em bitch don just cause money ass ba man come
Tópico: 2
love oh yeah baby know don ll come let got just girl like ooh hey
Tópico: 3
life god world die eyes death dead ll blood time away light black soul man
Tópico: 4
like got yo man shit know don nigga ya ain yeah cause niggas right rock
Tópico: 5
don just know want say got cause time like gonna right make ll ve think


In [50]:
fit_best_lda = best_lda.transform(count_vectorizer_fit)
topic_for_each_doc = pd.DataFrame(
    np.round(fit_best_lda, 3),
    index = df.index
)

In [56]:
topic_for_each_doc.columns = [f"Topico {x}" for x in range(1, best_lda.n_components +1)]

In [58]:
df_concat = pd.concat([df, topic_for_each_doc], axis=1)

In [60]:
df_concat['high_pr'] = np.argmax(topic_for_each_doc.values, axis = 1) + 1

In [61]:
df_concat

,Artis,Genre,Song,Lyrics,Topico 1,Topico 2,Topico 3,Topico 4,Topico 5,high_pr
0,Public Enemy,hiphop,You're Gonna Get Yours,"(Flavor Flav) \n Oh-oh Chuck, they out to get ...",0.001,0.001,0.039,0.431,0.528,5
1,Public Enemy,hiphop,Sophisticated Bitch,"That woman in the corner, cold playin' the rol...",0.198,0.001,0.072,0.271,0.459,5
2,Public Enemy,hiphop,Miuzi Weighs A Ton,"Yo Chuck, run a power move on them \n (Yeah) \...",0.010,0.001,0.066,0.777,0.146,4
3,Public Enemy,hiphop,Timebomb,"(Intro - Flavor Flav) \n Hey, Chuck, we got so...",0.043,0.041,0.001,0.651,0.265,4
4,Public Enemy,hiphop,Too Much Posse,(Intro - Flavor Flav) \n What do you got to sa...,0.179,0.066,0.111,0.481,0.162,4
5,Public Enemy,hiphop,Rightstarter (Message To A Black Man),"Mind over matter, mouth in motion \n Can't def...",0.039,0.001,0.182,0.123,0.656,5
6,Public Enemy,hiphop,Public Enemy No. 1,"(Intro: Flavor Flav) \n Yo Chuck, bust a move ...",0.030,0.001,0.001,0.579,0.390,4
7,Public Enemy,hiphop,M.P.E.,I'm cold gettin' busy while I'm shakin' you do...,0.119,0.001,0.126,0.564,0.189,4
8,Public Enemy,hiphop,Yo! Bum Rush The Show,"I am taking no prisoners, taking no shorts \n ...",0.213,0.074,0.050,0.438,0.225,4
9,Public Enemy,hiphop,Raise The Roof,"Raise your hands, so we can \n Raise the roof,...",0.001,0.001,0.304,0.448,0.247,4
